# Using the Indices data-set

The python version of this sample can be found [here](https://github.com/shyams80/plutons/blob/master/docs-py/Indices.ipynb)

In [3]:
library(DBI)
library(plutoDbR)
library(plutoR)
library(tidyverse)
options("scipen"=999)
source("/usr/share/pluto/config.R")

indices<-Indices()

### fetch the last 10 day India VIX levels

In [4]:
startDate <- Sys.Date() - 10
indices$IndiaVixTimeSeries() %>%
  filter(TIME_STAMP >= startDate) %>%
  arrange(TIME_STAMP) %>%
  print(n=Inf)

# Source:     lazy query [?? x 5]
# Database:   NORWAY:StockViz:R
# Ordered by: TIME_STAMP
  TIME_STAMP  HIGH   LOW  OPEN CLOSE
  <chr>      <dbl> <dbl> <dbl> <dbl>
1 2019-09-27  16.8  15.9  16.3  16.1
2 2019-09-30  16.5  15.8  16.1  15.9
3 2019-10-01  18.2  14.9  15.9  16.8
4 2019-10-03  18.0  16.4  16.8  17.7
5 2019-10-04  17.9  16.3  17.7  17.6


### fetch the latest rates across tenors

In [5]:
maxDt <- (indices$IndiaGsecTimeSeries() %>%
    summarize(MAX_TS = max(TIME_STAMP)) %>%
    collect())$MAX_TS[1]

indices$IndiaGsecTimeSeries() %>%
    filter(TIME_STAMP == maxDt) %>%
    arrange(NAME) %>%
    print()

# Source:     lazy query [?? x 7]
# Database:   NORWAY:StockViz:R
# Ordered by: NAME
  NAME  TIME_STAMP   TRI   PRI COUPON   YTM DURATION
  <chr> <chr>      <dbl> <dbl>  <dbl> <dbl>    <dbl>
1 0_5   2019-10-04 2985.  841.   7.87  6.05     2.75
2 10_15 2019-10-04 2954.  900.   7.79  6.94     8.14
3 15_20 2019-10-04 2957.  893.   7.74  6.99     9.59
4 20_30 2019-10-04 2940.  877.   8.05  7.14    11.5 
5 5_10  2019-10-04 2980.  861.   7.79  6.70     5.62


### get all total-return NSE indices for the latest time_stamp

In [6]:
maxDtNse <- (indices$NseTimeSeries() %>% summarize(MAX_TS = max(TIME_STAMP)) %>% collect())$MAX_TS[1]
indices$NseTimeSeries() %>%
     group_by(NAME) %>%
     filter(TIME_STAMP == maxDtNse && NAME %like% '% TR%') %>%
     select(NAME) %>%
     print(n = 10)

# Source:   lazy query [?? x 1]
# Database: NORWAY:StockViz:R
# Groups:   NAME
   NAME                                          
   <chr>                                         
 1 NIFTY 100 TR                                  
 2 NIFTY 200 TR                                  
 3 NIFTY 50 FUTURES TR INDEX                     
 4 NIFTY 50 TR                                   
 5 NIFTY 500 TR                                  
 6 NIFTY ADITYA BIRLA GROUP TR                   
 7 NIFTY ALPHA 50 TR                             
 8 NIFTY ALPHA LOW-VOLATILITY 30 TR              
 9 NIFTY ALPHA QUALITY LOW-VOLATILITY 30 TR      
10 NIFTY ALPHA QUALITY VALUE LOW-VOLATILITY 30 TR
# … with more rows


## get start and end dates for all total-return NSE indices

In [7]:
indices$NseTimeSeries() %>%
     group_by(NAME) %>%
     filter(NAME %like% '% TR%') %>%
     summarize(ST=min(TIME_STAMP), ET=max(TIME_STAMP)) %>%
     print(n = 10)

# Source:   lazy query [?? x 3]
# Database: NORWAY:StockViz:R
   NAME                                           ST         ET        
   <chr>                                          <chr>      <chr>     
 1 NIFTY 100 TR                                   2003-01-01 2019-10-04
 2 NIFTY 200 TR                                   2004-01-01 2019-10-04
 3 NIFTY 50 FUTURES TR INDEX                      2005-04-01 2019-10-04
 4 NIFTY 50 TR                                    1999-06-30 2019-10-04
 5 NIFTY 500 TR                                   1998-11-26 2019-10-04
 6 NIFTY ADITYA BIRLA GROUP TR                    2005-04-01 2019-10-04
 7 NIFTY ALPHA 50 TR                              2003-12-31 2019-10-04
 8 NIFTY ALPHA LOW-VOLATILITY 30 TR               2005-04-01 2019-10-04
 9 NIFTY ALPHA QUALITY LOW-VOLATILITY 30 TR       2005-04-01 2019-10-04
10 NIFTY ALPHA QUALITY VALUE LOW-VOLATILITY 30 TR 2005-04-01 2019-10-04
# … with more rows


### get all BSE indices for the latest time_stamp

In [8]:
maxDtBse <- (indices$BseTimeSeries() %>% summarize(MAX_TS = max(TIME_STAMP)) %>% collect())$MAX_TS[1]
indices$BseTimeSeries() %>%
     group_by(NAME) %>%
     filter(TIME_STAMP == maxDtBse) %>%
     select(NAME) %>%
     print(n = 10)

# Source:   lazy query [?? x 1]
# Database: NORWAY:StockViz:R
# Groups:   NAME
   NAME  
   <chr> 
 1 BSE100
 2 BSE200
 3 BSE500
 4 BSEMID
 5 BSESML
 6 CARBON
 7 CPSE  
 8 DOL100
 9 DOLL30
10 DOLLEX
# … with more rows


### fetch the latest NSE NIFTY 50 constituents 

In [9]:
maxDt <- (indices$NseConstituents() %>%
          filter(NAME == "NIFTY 50") %>%
          summarize(MAX_TS = max(TIME_STAMP)) %>% 
          collect())$MAX_TS[1]

indices$NseConstituents() %>%
     filter(TIME_STAMP == maxDt & NAME == "NIFTY 50") %>%
     print(n = 10)

# Source:   lazy query [?? x 5]
# Database: NORWAY:StockViz:R
   NAME     TIME_STAMP SYMBOL     INDUSTRY           CAP_WEIGHT
   <chr>    <chr>      <chr>      <chr>                   <dbl>
 1 NIFTY 50 2019-10-04 ADANIPORTS SERVICES                0.661
 2 NIFTY 50 2019-10-04 ASIANPAINT CONSUMER GOODS          1.61 
 3 NIFTY 50 2019-10-04 AXISBANK   FINANCIAL SERVICES      3.00 
 4 NIFTY 50 2019-10-04 BAJAJ-AUTO AUTOMOBILE              0.816
 5 NIFTY 50 2019-10-04 BAJAJFINSV FINANCIAL SERVICES      1.01 
 6 NIFTY 50 2019-10-04 BAJFINANCE FINANCIAL SERVICES      1.88 
 7 NIFTY 50 2019-10-04 BHARTIARTL TELECOM                 1.44 
 8 NIFTY 50 2019-10-04 BPCL       ENERGY                  0.733
 9 NIFTY 50 2019-10-04 BRITANNIA  CONSUMER GOODS          0.721
10 NIFTY 50 2019-10-04 CIPLA      PHARMA                  0.504
# … with more rows


### fetch the latest BSE SENSEX constituents    

In [10]:
maxDt <- (indices$BseConstituents() %>%
          filter(NAME == "sp bse sensex") %>%
          summarize(MAX_TS = max(TIME_STAMP)) %>% 
          collect())$MAX_TS[1]

indices$BseConstituents() %>%
     filter(TIME_STAMP == maxDt & NAME == "sp bse sensex") %>%
     print(n = 10)

# Source:   lazy query [?? x 5]
# Database: NORWAY:StockViz:R
   NAME          TIME_STAMP   CODE SYMBOL     SECURITY_NAME                   
   <chr>         <chr>       <int> <chr>      <chr>                           
 1 sp bse sensex 2019-07-01 500010 HDFC       Housing Development Finance Corp
 2 sp bse sensex 2019-07-01 500034 BAJFINANCE Bajaj Finance Ltd               
 3 sp bse sensex 2019-07-01 500112 SBIN       State Bank of India             
 4 sp bse sensex 2019-07-01 500180 HDFCBANK   HDFC Bank Ltd                   
 5 sp bse sensex 2019-07-01 500182 HEROMOTOCO Hero MotoCorp Ltd               
 6 sp bse sensex 2019-07-01 500209 INFY       Infosys Ltd                     
 7 sp bse sensex 2019-07-01 500247 KOTAKBANK  Kotak Mahindra Bank Ltd         
 8 sp bse sensex 2019-07-01 500295 VEDL       Vedanta Ltd                     
 9 sp bse sensex 2019-07-01 500312 ONGC       Oil & Natural Gas Corp Ltd      
10 sp bse sensex 2019-07-01 500325 RELIANCE   Reliance Industries Ltd

### fetch index date ranges published on yahoo finance

In [11]:
indices$YahooFinanceTimeSeries() %>%
    group_by(NAME) %>%
    summarize(ST = min(TIME_STAMP), ET = max(TIME_STAMP)) %>%
    arrange(NAME) %>%
    print(n=Inf)

# Source:     lazy query [?? x 3]
# Database:   NORWAY:StockVizUs2:R
# Ordered by: NAME
  NAME  ST         ET        
  <chr> <chr>      <chr>     
1 ^GSPC 1950-01-03 2019-08-09
2 ^MID  1981-01-02 2019-08-09
3 ^N225 1965-01-05 2019-08-09
4 ^NDX  1985-10-01 2019-08-09
5 ^RAG  2002-09-30 2019-08-09
6 ^RAV  2002-09-30 2019-08-09
7 ^RUA  1987-09-10 2019-08-09
8 ^RUT  1987-09-10 2019-08-09
9 ^VIX  1990-01-02 2019-08-09


### fetch index date ranges published by Barclays

In [12]:
indices$BarclaysMeta() %>%
  inner_join(indices$BarclaysTimeSeries()) %>%
  group_by(FAMILY, NAME) %>%
  summarize(ST = min(TIME_STAMP), ET = max(TIME_STAMP)) %>%
  select(FAMILY, NAME, ST, ET) %>%
  arrange(FAMILY) %>%
  print(n = Inf)

# Source:     lazy query [?? x 4]
# Database:   NORWAY:StockVizUs2:R
# Groups:     FAMILY
# Ordered by: FAMILY
    FAMILY                NAME                                   ST      ET     
    <chr>                 <chr>                                  <chr>   <chr>  
  1 ARMOUR                Barclays ARMOUR CHF 10% Index          2012-1… 2019-0…
  2 ARMOUR                Barclays ARMOUR CHF 7% Index           1998-1… 2019-0…
  3 ARMOUR                Barclays ARMOUR CHF Index              2012-0… 2019-0…
  4 ARMOUR                Barclays ARMOUR EUR 10% Index          2012-1… 2019-0…
  5 ARMOUR                Barclays ARMOUR EUR 7% Index           2012-1… 2019-0…
  6 ARMOUR                Barclays ARMOUR EUR Index              2012-0… 2019-0…
  7 ARMOUR                Barclays ARMOUR II Gross USD 5% Exces… 2001-1… 2019-0…
  8 ARMOUR                Barclays ARMOUR II Gross USD 5% Total… 2001-1… 2019-0…
  9 ARMOUR                Barclays ARMOUR II Gross USD 7% Exces… 2001-1… 2019-0

### fetch index date ranges published by Wilshire

In [13]:
indices$WilshireMeta() %>%
  inner_join(indices$WilshireTimeSeries()) %>%
  group_by(NAME) %>%
  summarize(ST = min(TIME_STAMP), ET = max(TIME_STAMP)) %>%
  select(NAME, ST, ET) %>%
  arrange(ST) %>%
  print(n = Inf)

# Source:     lazy query [?? x 3]
# Database:   NORWAY:StockVizUs2:R
# Ordered by: ST
   NAME                                               ST         ET        
   <chr>                                              <chr>      <chr>     
 1 Wilshire 5000 (Full Cap) Total Market              1970-12-31 2019-10-04
 2 Wilshire 5000 Total Market                         1970-12-31 2019-10-04
 3 Wilshire 5000 Equal Weighted                       1970-12-31 2019-09-30
 4 Wilshire US Large-Cap Equal Weight                 1977-12-30 2019-10-04
 5 Wilshire US REIT                                   1977-12-30 2019-10-04
 6 Wilshire US RESI                                   1977-12-30 2019-10-04
 7 Wilshire US Large-Cap                              1978-06-30 2019-10-04
 8 Wilshire US Large-Cap Growth                       1978-06-30 2019-10-04
 9 Wilshire US Large-Cap Value                        1978-06-30 2019-10-04
10 Wilshire US Mid-Cap                                1978-06-30 2019-10-04
11

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)